In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout


Pegando as images de treino

In [4]:
data = []
labels = []
classes = 43
cur_path = os.getcwd()
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

data = np.array(data)
labels = np.array(labels)

Separando Dataset para treino e test

Converterndo para one hot encoding

In [5]:
print(data.shape, labels.shape)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

(39209, 30, 30, 3) (39209,)
(31367, 30, 30, 3) (7842, 30, 30, 3) (31367,) (7842,)


In [10]:
# plt.figure()
# plt.imshow(X_train[70])
# plt.colorbar()
# plt.grid(False)
# plt.show()

Criando o modelo

In [6]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))


(30, 30, 3)


Compilando o modelo e fazendo o Fit

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 100
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))
model.save("my_model.h5")

Epoch 1/5
981/981 [==============================] - 7s 7ms/step - loss: 1.5986 - accuracy: 0.6991 - val_loss: 0.4189 - val_accuracy: 0.8883
Epoch 2/5
981/981 [==============================] - 7s 7ms/step - loss: 0.2920 - accuracy: 0.9228 - val_loss: 0.2792 - val_accuracy: 0.9271
Epoch 3/5
981/981 [==============================] - 7s 7ms/step - loss: 0.1714 - accuracy: 0.9550 - val_loss: 0.2599 - val_accuracy: 0.9397
Epoch 4/5
981/981 [==============================] - 7s 7ms/step - loss: 0.1574 - accuracy: 0.9607 - val_loss: 0.1991 - val_accuracy: 0.9546
Epoch 5/5
981/981 [==============================] - 7s 7ms/step - loss: 0.1400 - accuracy: 0.9647 - val_loss: 0.2295 - val_accuracy: 0.9518


Plotando em graficos


In [ ]:
# plt.figure(0)
# plt.plot(history.history['accuracy'], label='training accuracy')
# plt.plot(history.history['val_accuracy'], label='val accuracy')
# plt.title('Accuracy')
# plt.xlabel('epochs')
# plt.ylabel('accuracy')
# plt.legend()
# plt.show()

In [ ]:
# plt.figure(1)
# plt.plot(history.history['loss'], label='training loss')
# plt.plot(history.history['val_loss'], label='val loss')
# plt.title('Loss')
# plt.xlabel('epochs')
# plt.ylabel('loss')
# plt.legend()
# plt.show()

Dando Load no modelo

In [ ]:
model = load_model("my_model.h5")

Testando a acuracia com o dataset de test

In [18]:
y_test = pd.read_csv('input/Train.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]
for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))
X_test=np.array(data)
predict_x=model.predict(X_test) 
model.save('traffic_classifier.h5')

1226/1226 [==============================] - 3s 3ms/step
